In [48]:
import os
import cv2
import csv
import numpy as np

Download reference/compressed images

In [34]:
if not (os.path.exists("study_images_crop_all.zip") or os.path.isdir("study_images_crop_all")):
  !wget -O study_images_crop_all.zip https://utexas.box.com/shared/static/izd3wopcohce7htgox0cb55v5netni7l.zip
  !unzip study_images_crop_all.zip
  !rm -r study_images_crop_all.zip __MACOSX

Download result csv file of subjective study

In [36]:
if not os.path.exists("study_results.csv"):
  !wget -O study_results.csv https://utexas.box.com/shared/static/5ywjy089mgg0v02qqy3mksvs9tt9e1mu.csv

# Demo: 2AFC score of PSNR_rgb

In [52]:
IMG_PATH = './study_images_crop_all'
psnr_rgb_2afc_all = []
with open('study_results.csv') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # PSNR_rgb
        img_ref  = cv2.imread(os.path.join(IMG_PATH, row['ref_name']))
        img_base = cv2.imread(os.path.join(IMG_PATH, row['base_name']))
        img_ours = cv2.imread(os.path.join(IMG_PATH, row['ours_name']))
        psnr_base = cv2.PSNR(img_ref, img_base)
        psnr_ours = cv2.PSNR(img_ref, img_ours)
        # Human subject score
        winrate_num = int(row['num_vote_ours'])
        winrate_den = int(row['num_vote_ours']) + int(row['num_vote_base'])
        assert winrate_den == 47
        winrate = winrate_num / winrate_den
        # 2AFC
        score_2afc = winrate*(psnr_ours > psnr_base) + \
                     (1-winrate)*(1-(psnr_ours > psnr_base))
        psnr_rgb_2afc_all.append(score_2afc)
print(f"2AFC score of PSNR_rgb is {np.mean(psnr_rgb_2afc_all)}")


2AFC score of PSNR_rgb is 0.6127364066193853


# Demo: 2AFC score of VIFp
We used the pixel domain VIF implementation, which can be found here: https://live.ece.utexas.edu/research/Quality/VIF.htm

In [53]:
IMG_PATH = './study_images_crop_all'
vifp_2afc_all = []
with open('study_results.csv') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # Pre-calculated VIFp scores read from csv 
        vifp_base = float(row['vifp_base'])
        vifp_ours = float(row['vifp_ours'])
        # Human subject score
        winrate_num = int(row['num_vote_ours'])
        winrate_den = int(row['num_vote_ours']) + int(row['num_vote_base'])
        assert winrate_den == 47
        winrate = winrate_num / winrate_den
        # 2AFC
        score_2afc = winrate*(vifp_ours > vifp_base) + \
                     (1-winrate)*(1-(vifp_ours > vifp_base))
        vifp_2afc_all.append(score_2afc)

print(f"2AFC score of VIFp is {np.mean(vifp_2afc_all)}")

2AFC score of VIFp is 0.6589834515366431
